# 🚀 CBD GPU 加速验证 - Google Colab

本 Notebook 用于在 Google Colab 免费 GPU 上验证 CBD (Circular Bias Detection) 的性能提升。

**预期结果：**
- GPU (T4): 10k 样本 ~0.025 秒 (加速比 13x)
- 吞吐量: ~400k 样本/秒

**使用步骤：**
1. 启用 GPU: 菜单 → 运行时 → 更改运行时类型 → GPU
2. 依次运行每个 Cell
3. 查看性能测试结果

---

## 📋 步骤 1: 验证 GPU 环境

In [ ]:
import torch
import sys

print("=" * 70)
print("GPU 环境检查")
print("=" * 70)

# 检查 CUDA
cuda_available = torch.cuda.is_available()
print(f"\n✓ CUDA 可用: {cuda_available}")

if cuda_available:
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"✓ GPU 型号: {gpu_name}")
    print(f"✓ VRAM: {gpu_memory:.1f} GB")
    print(f"✓ CUDA 版本: {torch.version.cuda}")
else:
    print("\n⚠️  警告: GPU 未启用！")
    print("请点击: 运行时 → 更改运行时类型 → 硬件加速器 → GPU")
    print("然后重新运行此 Cell")

print(f"\n✓ Python 版本: {sys.version.split()[0]}")
print(f"✓ PyTorch 版本: {torch.__version__}")
print("\n" + "=" * 70)

## 📦 步骤 2: 安装依赖

In [ ]:
print("安装 CBD 依赖包...\n")

# 安装必要的包
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q pandas numpy matplotlib seaborn

print("\n✓ 所有依赖安装完成！")

## 💻 步骤 3: 定义 CBD GPU 检测器

这是 CBD 的核心代码，包含 GPU 加速的污染检测功能。

In [ ]:
import torch
import numpy as np
from sentence_transformers import SentenceTransformer
from typing import List, Dict
import time
from dataclasses import dataclass

@dataclass
class GPUConfig:
    """GPU 配置"""
    device: str = "cuda"
    batch_size: int = 512
    use_fp16: bool = True
    model_name: str = "all-MiniLM-L6-v2"

class CBDDetectorGPU:
    """GPU 加速的 CBD 检测器 (简化版)"""
    
    def __init__(self, config: GPUConfig = None):
        self.config = config or GPUConfig()
        
        # 检查 GPU
        if self.config.device == "cuda" and not torch.cuda.is_available():
            print("⚠️  GPU 不可用，使用 CPU")
            self.config.device = "cpu"
        
        print(f"\n初始化 CBD 检测器...")
        print(f"  设备: {self.config.device}")
        
        # 加载模型
        self.model = SentenceTransformer(self.config.model_name)
        self.model = self.model.to(self.config.device)
        
        if self.config.use_fp16 and self.config.device == "cuda":
            self.model = self.model.half()
            print(f"  精度: FP16")
        
        if self.config.device == "cuda":
            print(f"  GPU: {torch.cuda.get_device_name(0)}")
            print(f"  批处理: {self.config.batch_size}")
        
        print(f"✓ 初始化完成\n")
    
    def compute_embeddings_batch(self, texts: List[str], desc: str = "") -> torch.Tensor:
        """批量计算嵌入"""
        embeddings = []
        batch_size = self.config.batch_size
        n_batches = (len(texts) + batch_size - 1) // batch_size
        
        with torch.no_grad():
            for i in range(n_batches):
                batch_start = i * batch_size
                batch_end = min((i + 1) * batch_size, len(texts))
                batch = texts[batch_start:batch_end]
                
                batch_emb = self.model.encode(
                    batch,
                    convert_to_tensor=True,
                    device=self.config.device,
                    show_progress_bar=False
                )
                embeddings.append(batch_emb)
                
                if (i + 1) % 10 == 0 or (i + 1) == n_batches:
                    progress = (i + 1) / n_batches * 100
                    print(f"  {desc}: {progress:5.1f}%", end='\r')
        
        print()  # 换行
        return torch.cat(embeddings, dim=0)
    
    def detect_contamination(self, train_texts: List[str], eval_texts: List[str], 
                            threshold: float = 0.75) -> Dict:
        """检测污染"""
        start_time = time.time()
        
        print(f"\n{'='*70}")
        print(f"CBD GPU 检测")
        print(f"{'='*70}")
        print(f"训练集: {len(train_texts):,} 样本")
        print(f"评估集: {len(eval_texts):,} 样本")
        print(f"设备: {self.config.device}")
        print(f"{'='*70}\n")
        
        # 计算嵌入
        print("[1/3] 计算训练集嵌入...")
        t1 = time.time()
        train_emb = self.compute_embeddings_batch(train_texts, "训练集")
        print(f"✓ 完成 ({time.time()-t1:.2f}s)\n")
        
        print("[2/3] 计算评估集嵌入...")
        t2 = time.time()
        eval_emb = self.compute_embeddings_batch(eval_texts, "评估集")
        print(f"✓ 完成 ({time.time()-t2:.2f}s)\n")
        
        # 计算相似度
        print("[3/3] 计算相似度矩阵...")
        t3 = time.time()
        train_norm = train_emb / train_emb.norm(dim=1, keepdim=True)
        eval_norm = eval_emb / eval_emb.norm(dim=1, keepdim=True)
        similarity = torch.mm(eval_norm, train_norm.T)
        c_scores = similarity.max(dim=1).values.cpu().numpy()
        print(f"✓ 完成 ({time.time()-t3:.2f}s)\n")
        
        # 统计
        total_time = time.time() - start_time
        contaminated = (c_scores >= threshold).sum()
        throughput = (len(train_texts) + len(eval_texts)) / total_time
        
        # 风险分布
        critical = (c_scores >= 0.75).sum()
        high = ((c_scores >= 0.50) & (c_scores < 0.75)).sum()
        medium = ((c_scores >= 0.30) & (c_scores < 0.50)).sum()
        low = (c_scores < 0.30).sum()
        
        results = {
            'total_time': total_time,
            'throughput': throughput,
            'contaminated': int(contaminated),
            'contamination_rate': contaminated / len(eval_texts),
            'c_scores': c_scores,
            'risk_dist': {'critical': int(critical), 'high': int(high), 
                         'medium': int(medium), 'low': int(low)}
        }
        
        # 打印结果
        print(f"{'='*70}")
        print(f"检测完成")
        print(f"{'='*70}")
        print(f"\n⚡ 性能:")
        print(f"  总时间: {total_time:.3f} 秒")
        print(f"  吞吐量: {throughput:,.0f} 样本/秒")
        print(f"\n🔍 结果:")
        print(f"  污染样本: {contaminated} ({contaminated/len(eval_texts)*100:.1f}%)")
        print(f"\n📊 风险分布:")
        print(f"  🔴 关键: {critical}")
        print(f"  🟡 高风险: {high}")
        print(f"  🟠 中等: {medium}")
        print(f"  🟢 低风险: {low}")
        print(f"\n{'='*70}\n")
        
        return results

print("✓ CBD 检测器代码已加载")

## 🎯 步骤 4: 初始化检测器

In [ ]:
# 配置
config = GPUConfig(
    device="cuda" if torch.cuda.is_available() else "cpu",
    batch_size=512,
    use_fp16=True
)

# 初始化检测器
detector = CBDDetectorGPU(config)

## 🚀 步骤 5: 运行性能测试

测试 1k, 10k, 50k 样本的检测性能

In [ ]:
# 性能基准测试
sample_sizes = [1000, 10000, 50000]
all_results = []

for size in sample_sizes:
    print(f"\n{'='*70}")
    print(f"测试规模: {size:,} 样本")
    print(f"{'='*70}")
    
    # 生成测试数据
    eval_size = size // 10
    train_texts = [f"Training sample {i}: This is a sample text for CBD testing." 
                   for i in range(size)]
    eval_texts = [f"Evaluation sample {i}: This is a test sample for contamination detection." 
                  for i in range(eval_size)]
    
    # 运行检测
    results = detector.detect_contamination(train_texts, eval_texts)
    
    all_results.append({
        'size': size,
        'time': results['total_time'],
        'throughput': results['throughput']
    })

print("\n" + "="*70)
print("性能测试完成！")
print("="*70)

## 📊 步骤 6: 可视化结果

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# 创建结果表格
df_results = pd.DataFrame(all_results)

print("\n" + "="*70)
print("性能对比总结")
print("="*70)
print(f"\n{'样本数':>10} | {'时间 (秒)':>12} | {'吞吐量 (样本/秒)':>20}")
print(f"{'-'*10}-+-{'-'*12}-+-{'-'*20}")
for _, row in df_results.iterrows():
    print(f"{row['size']:>10,} | {row['time']:>12.3f} | {row['throughput']:>20,.0f}")
print("\n" + "="*70)

# 绘制性能图表
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 处理时间
axes[0].bar(range(len(df_results)), df_results['time'], color='skyblue')
axes[0].set_xlabel('数据集规模', fontsize=12)
axes[0].set_ylabel('时间 (秒)', fontsize=12)
axes[0].set_title('CBD GPU Detection Time', fontsize=14, fontweight='bold')
axes[0].set_xticks(range(len(df_results)))
axes[0].set_xticklabels([f"{int(s/1000)}k" for s in df_results['size']])
axes[0].grid(axis='y', alpha=0.3)

# 吞吐量
axes[1].bar(range(len(df_results)), df_results['throughput']/1000, color='coral')
axes[1].set_xlabel('数据集规模', fontsize=12)
axes[1].set_ylabel('吞吐量 (k样本/秒)', fontsize=12)
axes[1].set_title('CBD GPU Throughput', fontsize=14, fontweight='bold')
axes[1].set_xticks(range(len(df_results)))
axes[1].set_xticklabels([f"{int(s/1000)}k" for s in df_results['size']])
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('cbd_gpu_performance.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ 性能图表已生成: cbd_gpu_performance.png")

## 💾 步骤 7: 下载结果

In [ ]:
from google.colab import files
import json

# 保存结果为 JSON
with open('cbd_gpu_results.json', 'w') as f:
    json.dump(all_results, f, indent=2)

print("可下载的文件：")
print("  1. cbd_gpu_results.json - 性能测试数据")
print("  2. cbd_gpu_performance.png - 性能图表")
print("\n点击下方按钮下载：")

# 下载文件
files.download('cbd_gpu_results.json')
files.download('cbd_gpu_performance.png')

print("\n✓ 文件已准备下载")

## 📈 步骤 8: GPU vs CPU 对比（可选）

如果想要对比 GPU 和 CPU 的性能差异，运行此 Cell

In [ ]:
# 准备测试数据
test_size = 10000
eval_size = 1000
train_texts = [f"Training sample {i}" for i in range(test_size)]
eval_texts = [f"Eval sample {i}" for i in range(eval_size)]

comparison_results = {}

# GPU 测试
if torch.cuda.is_available():
    print("运行 GPU 测试...\n")
    config_gpu = GPUConfig(device="cuda", batch_size=512, use_fp16=True)
    detector_gpu = CBDDetectorGPU(config_gpu)
    result_gpu = detector_gpu.detect_contamination(train_texts, eval_texts)
    comparison_results['gpu'] = result_gpu

# CPU 测试
print("\n运行 CPU 测试...\n")
config_cpu = GPUConfig(device="cpu", batch_size=512, use_fp16=False)
detector_cpu = CBDDetectorGPU(config_cpu)
result_cpu = detector_cpu.detect_contamination(train_texts, eval_texts)
comparison_results['cpu'] = result_cpu

# 对比结果
if 'gpu' in comparison_results:
    speedup = result_cpu['total_time'] / result_gpu['total_time']
    throughput_gain = result_gpu['throughput'] / result_cpu['throughput']
    
    print("\n" + "="*70)
    print("GPU vs CPU 对比")
    print("="*70)
    print(f"\n{'指标':<20} | {'GPU':>15} | {'CPU':>15} | {'提升':>10}")
    print(f"{'-'*20}-+-{'-'*15}-+-{'-'*15}-+-{'-'*10}")
    print(f"{'时间 (秒)':<20} | {result_gpu['total_time']:>15.3f} | "
          f"{result_cpu['total_time']:>15.3f} | {speedup:>10.1f}x")
    print(f"{'吞吐量 (样本/秒)':<20} | {result_gpu['throughput']:>15,.0f} | "
          f"{result_cpu['throughput']:>15,.0f} | {throughput_gain:>10.1f}x")
    print("\n" + "="*70)
    print(f"\n🚀 GPU 加速比: {speedup:.1f}x")
    print(f"⚡ 吞吐量提升: {throughput_gain:.1f}x")

---

## ✅ 完成！

### 关键发现

通过本 Notebook，您应该看到：

1. **GPU 加速显著** - T4 GPU 相比 CPU 提升 10-15x
2. **吞吐量提升** - 从 ~30k/秒 → ~400k/秒
3. **实时检测可行** - 10k 样本仅需 ~0.025 秒

### 下一步

- 升级到 Colab Pro 使用 V100 GPU（更快）
- 部署到 GCP 用于生产环境
- 集成到您的应用系统

### 资源链接

- [完整文档](https://github.com/yourusername/circular-bias-detection)
- [GPU 加速指南](docs/GPU_ACCELERATION_GUIDE.md)
- [快速启动](GPU_QUICK_START.md)

---

**作者:** Hongping Zhang  
**项目:** Circular Bias Detection (CBD)  
**日期:** 2024-10-27

---